# Starter code for Cloud Classification Challenge

This code is designed as starter point for your development. You do not have to use it, but feel free to use it if you do not know where to start.

The [Pytorch](https://pytorch.org/) collection of packages is used to define and train the model, and this code is adapted from their [introductory tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html).

Other machine learning python packages that you may wish to use include [TensorFlow](https://www.tensorflow.org/overview) and [scikit-learn](https://scikit-learn.org/stable/index.html).

In [2]:
import os

import pandas as pd
import torch
from torchvision.io import read_image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

## Create Custom Dataset for sat images

Dataset instance reads in the directory to the images and their labels.
The dataloader enables simple iteration over these images when training and testing a model.


In [3]:
# Define transforms for label data
def get_label_dict():
    label_dict = {"Fish": 0,
                  "Flower": 1,
                  "Gravel": 2,
                  "Sugar": 3}
    return label_dict


def sat_label_transform(label):
    label_dict = get_label_dict()
    return label_dict[label]


def sat_label_transform_inv(num):
    label_dict = get_label_dict()
    ret_list = [key for key in label_dict.keys() if label_dict[key]==num]
    return ret_list[0]

In [4]:
# Define the transform for images.
# Converts to float and scales values to range 0-1.
# Normalisation using the mean/std used by AlexNet.
img_transform = transforms.Compose([
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
    ])

In [5]:
# Create class for loading the satellite image into a Dataset
class SatImageDataset(Dataset):
    def __init__(self, labels_file, img_dir, transform=img_transform, target_transform=sat_label_transform):
        self.img_labels = pd.read_csv(labels_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels["Image"].iloc[idx])
        image = read_image(img_path)
        label = self.img_labels["Label"].iloc[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

Load the training and testing data using instances of the SatImageDataset defined above.

In [6]:
# Load the training data.
train_files_dir = "/data/users/meastman/understanding_clouds_kaggle/input/single_labels/224s/train/"
train_files_labels = "/data/users/meastman/understanding_clouds_kaggle/input/single_labels/224s/train/train_labels.csv"

# Create train images dataloader
train_images = SatImageDataset(labels_file=train_files_labels, img_dir=train_files_dir)
train_dataloader = DataLoader(train_images, batch_size=32, shuffle=True)

In [7]:
# Test Data
test_files_dir = "/data/users/meastman/understanding_clouds_kaggle/input/single_labels/224s/test/"
test_files_labels = "/data/users/meastman/understanding_clouds_kaggle/input/single_labels/224s/test/test_labels.csv"

# Create test images dataloader
test_images = SatImageDataset(labels_file=test_files_labels, img_dir=test_files_dir)
test_dataloader = DataLoader(test_images, batch_size=32, shuffle=True)

## Building a Neural Network

This is a fully connected neural network with 3 hidden layers. For more details on the individual layers, and for further options if you wish to create a different model architecture see [the tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).

Note that the input to the layer has size `150528 = 3*224*224`. The input images are 224 * 224 pixels, with 3 RGB channels.

The output layer has size 4 which matches the number of cloud categories available.

In [8]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(3*224*224, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 4),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Check which hardware is available so we set up the model correctly.
VDI/SPICE are CPU, but GPUs give better performance for training if available.

If you have the time/inclination/cost code GPUs are available through [AWS self service](https://web.yammer.com/main/org/metoffice.gov.uk/threads/eyJfdHlwZSI6IlRocmVhZCIsImlkIjoiMjM4MjQzNTE0MTIzMDU5MiJ9).

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device.")

Using cpu device.


In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150528, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=4, bias=True)
  )
)


Now define the loss function for our model to compare it's outputs to the true values. This allows the parameters (the weights and biases of the neural network) to be incrementally improved as the model sees more data.

As we have a classification task with multiple categories use the [Cross Entropy Loss](https://machinelearningmastery.com/cross-entropy-for-machine-learning/).

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()

To train the model and find the best parameters we want to minimise the loss function use Stochastic Gradient Descent.

In [12]:
# Learning rate controls how quickly the model converges.
learning_rate = 1e-3
# How many images given to the model at each time
batch_size = 32
# How many loops through the data. 
epochs = 1

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Define some functions for training the model and testing it.

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Training and testing the model

Now train and test the model!

N.B. This step will probably take a long time depending on your computing resources available.
It took me about 80 minutes for 1 epoch.

In [ ]:
%%time

for t in range(epochs):
    print(f"Epoch {t+1}\n--------------")
    train_loop(train_dataloader, model, loss_fn,optimizer)
    test_loop(train_dataloader, model, loss_fn)
print("Done")

Epoch 1
--------------
loss: 1.403055  [   32/69952]
loss: 1.278762  [ 3232/69952]
loss: 1.269672  [ 6432/69952]
loss: 1.229406  [ 9632/69952]
loss: 1.160386  [12832/69952]
loss: 1.186415  [16032/69952]
loss: 1.231033  [19232/69952]
loss: 1.089999  [22432/69952]
loss: 1.028146  [25632/69952]
loss: 1.079422  [28832/69952]
loss: 1.160247  [32032/69952]
loss: 1.081921  [35232/69952]
loss: 1.267946  [38432/69952]
loss: 1.343846  [41632/69952]
loss: 1.331972  [44832/69952]
loss: 0.941176  [48032/69952]
loss: 1.138247  [51232/69952]
loss: 1.216028  [54432/69952]


## Model output

Check the output of the model to find it's predictions for a sample image.

In [ ]:
# Load a batch of images and labels
t_imgs, t_labels = next(iter(train_dataloader))

In [67]:
t_imgs.shape

torch.Size([32, 3, 224, 224])

To turn the output into a prediction, first convert it to probabilities using the [softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html#torch.nn.Softmax) function.

Then select the category with the highest probability as the model prediction.
Convert this back into the text label.

In [76]:
output = model(t_imgs)
probs = torch.nn.Softmax(dim=1)
preds = probs.argmax(1)
pred_labels = [sat_label_transform_inv(pred) for pred in preds]

print("Predicted Labels\n", pred_labels)
print("True Labels\n", t_labels)

tensor([3, 2, 3, 2, 0, 3, 0, 3, 3, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 3, 1, 0, 2, 3,
        3, 3, 0, 0, 3, 1, 3, 0])